# Langchain4j example

Start by adding the dependencies.

In [2]:
//DEPS dev.langchain4j:langchain4j-open-ai:1.0.0
//DEPS dev.langchain4j:langchain4j:1.0.0
//DEPS org.slf4j:slf4j-simple:2.0.17

//DEPS io.github.cdimascio:dotenv-java:3.2.0


Adding to classpath: [/Users/max/.m2/repository/dev/langchain4j/langchain4j-open-ai/1.0.0/langchain4j-open-ai-1.0.0.jar, /Users/max/.m2/repository/dev/langchain4j/langchain4j-core/1.0.0/langchain4j-core-1.0.0.jar, /Users/max/.m2/repository/org/jspecify/jspecify/1.0.0/jspecify-1.0.0.jar, /Users/max/.m2/repository/dev/langchain4j/langchain4j-http-client/1.0.0/langchain4j-http-client-1.0.0.jar, /Users/max/.m2/repository/dev/langchain4j/langchain4j-http-client-jdk/1.0.0/langchain4j-http-client-jdk-1.0.0.jar, /Users/max/.m2/repository/com/fasterxml/jackson/core/jackson-annotations/2.19.0/jackson-annotations-2.19.0.jar, /Users/max/.m2/repository/com/fasterxml/jackson/core/jackson-core/2.19.0/jackson-core-2.19.0.jar, /Users/max/.m2/repository/com/fasterxml/jackson/core/jackson-databind/2.19.0/jackson-databind-2.19.0.jar, /Users/max/.m2/repository/com/knuddels/jtokkit/1.1.0/jtokkit-1.1.0.jar, /Users/max/.m2/repository/dev/langchain4j/langchain4j/1.0.0/langchain4j-1.0.0.jar, /Users/max/.m2/repo

## Setup env to get API secrets

In [11]:
import static java.lang.IO.*;
import java.nio.file.*;
import io.github.cdimascio.dotenv.Dotenv;

if(!Files.exists(Paths.get("env.properties"))) {
    println("env.properties not found, please create it and add OPENAI_API_KEY");
}

var env = Dotenv.configure().ignoreIfMissing().filename("env.properties").load();
var apiKey = env.get("OPENAI_API_KEY");

Setup language model:

In [12]:
import dev.langchain4j.model.openai.OpenAiChatModel;
import static java.time.Duration.ofSeconds;
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.memory.chat.MessageWindowChatMemory;

var chatModel = OpenAiChatModel.builder()
                .apiKey(apiKey)
                .modelName("gpt-4o-mini")
                .temperature(0.0)
                .timeout(ofSeconds(60))
                .build();

interface Assistant {
    String chat(String message);
}

var ai = AiServices.builder(Assistant.class)
                .chatModel(chatModel)
                .chatMemory(MessageWindowChatMemory.withMaxMessages(20))
                .build();

Lets talk with the AI:

In [45]:
display(ai.chat("generate a nice html paragraph with some nice colors and words, emojis and something moving/animating. only return the html, no other text"),"text/html");


```html
<p style="color: #FF6F61; font-size: 20px; font-family: 'Verdana', sans-serif; background-color: #FFF3E0; padding: 15px; border-radius: 10px; text-align: center;">
    🌟 Welcome to the world of <strong style="color: #FF8C00;">creativity</strong>! 🌈<br>
    Here, ideas bloom like flowers in spring. 🌸<br>
    <span style="display: inline-block; animation: bounce 1s infinite;">✨</span>
    <span style="display: inline-block; animation: bounce 1s infinite; animation-delay: 0.2s;">💡</span>
    <span style="display: inline-block; animation: bounce 1s infinite; animation-delay: 0.4s;">🎨</span>
    <br>
    Let's create something beautiful together! 🌻
</p>

<style>
    @keyframes bounce {
        0%, 20%, 50%, 80%, 100% {
            transform: translateY(0);
        }
        40% {
            transform: translateY(-20px);
        }
        60% {
            transform: translateY(-10px);
        }
    }
</style>
```

0c463db1-c39e-461f-8018-9358e110a9f3

## LLM's has limits

For example if we ask it to report how many letters are in a word, it will most likely not get it right.

In [28]:
ai.chat("how many r's are in in the word 'strawberry'?");

The word "strawberry" contains 2 "r's."

## Use a tool

Lets make a class that has methods that can be used as tools, i.e. to use for counting characters in a word.

In [ ]:
public class Tools {
   
    @Tool
    public long countOccurrences(String word, String character) {
        println("countOccurrences: " + word + " " + character); // put here so we can see the tool is called
        return word.chars().filter(c -> c == character.charAt(0)).count();
    }
}

new Tools().countOccurrences("strawberry", "r");

countOccurrences: strawberry r


3

In [41]:
var ai = AiServices.builder(Assistant.class)
                .chatModel(chatModel)
                .chatMemory(MessageWindowChatMemory.withMaxMessages(20))
                .tools(new Tools())
                .build();

Now lets ask again...

In [43]:
ai.chat("how many r's are in in the word 'strawberry'?");

There are 3 'r's in the word "strawberry."